# EfficientNetB0 Transfer Learning Example in Google Colab
This notebook demonstrates how to use EfficientNetB0 with Transfer Learning on CIFAR-10 dataset.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt

In [2]:
mixed_precision.set_global_policy('mixed_float16')

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
IMG_SIZE = 224
BATCH = 8
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
def make_dataset(images, labels, training=True):
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    if training:
        ds = ds.shuffle(5000)
    def _prep(img, lbl):
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32)
        img = preprocess_input(img)
        return img, lbl
    ds = ds.map(_prep, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

In [6]:
train_ds = make_dataset(x_train, y_train, training=True)
val_ds   = make_dataset(x_test, y_test, training=False)

In [ ]:
import numpy as np

def stratified_sample(x, y, samples_per_class=100):
    classes = np.argmax(y, axis=1)  # if y is one-hot
    selected_idx = []

    for c in np.unique(classes):
        class_idx = np.where(classes == c)[0]
        if len(class_idx) < samples_per_class:
            selected_idx.extend(class_idx.tolist())  # take all if less than needed
        else:
            chosen = np.random.choice(class_idx, samples_per_class, replace=False)
            selected_idx.extend(chosen.tolist())
    selected_idx = np.array(selected_idx)
    return x[selected_idx], y[selected_idx]

# Example usage:
x_train_small, y_train_small = stratified_sample(x_train, y_train, samples_per_class=100)
x_test_small, y_test_small = stratified_sample(x_test, y_test, samples_per_class=20)

# Then make your dataset with these smaller sets
train_ds = make_dataset(x_train_small, y_train_small, training=True)
val_ds = make_dataset(x_test_small, y_test_small, training=False)


In [7]:
base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax', dtype='float32')
])

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,829 (16.08 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(train_ds, epochs=3, validation_data=val_ds)

In [ ]:
loss, acc = model.evaluate(val_ds)
print(f"EfficientNetB0 test acc: {acc:.4f}")